# Footprint functionality
Here we show how to use the footprint functionality. It is not used directly in the matching, but can be applied on the recovery rates computation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pylab as plt
import healpy as hp

## Random data example

For display purposes, we will generate some quick random data to be used with the footprints.

In [ ]:
# For reproducibility
np.random.seed(1)

In [ ]:
from support import gen_cluster
input1, input2 = gen_cluster(ra_min=0, ra_max=30, dec_min=9, dec_max=30)

In [ ]:
from clevar import ClCatalog
tags = dict(ra='RA', dec='DEC', z='Z', mass='MASS',
            mass_err='MASS_ERR', z_err='Z_ERR',
            radius='RADIUS_ARCMIN')

c1 = ClCatalog('Cat1', data=input1, tags=tags, radius_unit='arcmin')
c2 = ClCatalog('Cat2', data=input2, tags=tags, radius_unit='arcmin')

# Format for nice display
for c in ('ra', 'dec', 'z', 'z_err', 'radius'):
    c1[c].info.format = '.2f'
    c2[c].info.format = '.2f'
for c in ('mass', 'mass_err'):
    c1[c].info.format = '.2e'
    c2[c].info.format = '.2e'

### Check position in healpix
Check what are the pixels that contain the clusters

In [ ]:
nside = 32
pixels1 = hp.ang2pix(nside, c1['ra'], c1['dec'], lonlat=True)
pixels2 = hp.ang2pix(nside, c2['ra'], c2['dec'], lonlat=True)

Plot to confirm selected pixels are correct

In [ ]:
from matplotlib import cm
import copy
cmap = copy.copy(cm.jet)
cmap.set_under('.1')
gcol = lambda cmap, level: '#{:02x}{:02x}{:02x}{:02x}'.format(*cmap(level,bytes=True))

# Map with pixels of each catalog
map_ = np.zeros(hp.nside2npix(nside))
map_[pixels1] += 1
map_[pixels2] += 2
map_[map_==0] = np.nan

f = plt.figure(figsize=(10, 10))
hp.cartview(map_, hold=True, latra=[5, 35], lonra=[-5, 40], cmap=cmap, cbar=False, flip='geo')
ax = f.axes[0]
ax.axis('on')
ax.scatter(c1['ra'], c1['dec'], s=5, label='Cat 1 clusters')
ax.scatter(c2['ra'], c2['dec'], s=5, label='Cat 2 clusters')

ax.plot(0, 0, zorder=0, color=gcol(cmap, 0.0), label='Footptint - Cat1 only')
ax.plot(0, 0, zorder=0, color=gcol(cmap, 0.5), label='Footptint - Cat2 only')
ax.plot(0, 0, zorder=0, color=gcol(cmap, 1.0), label='Footptint - BOTH')
ax.legend(loc=3)

## The Footprint object

ClEvaR uses the `Footprint` object to handle operations related to spatial masking. It has the following internal attributes:
- `data`: Table with main footprint data (ex: pixel, detfrac, zmax)
- `tags`: Dictionary that tells which are the default columns to be used
- `size`: Number of pixels in the catalog
- `pixel_dict`: Dictionary of indicies given the object pixel

In [ ]:
from clevar.footprint import Footprint

In [ ]:
from clevar.footprint.footprint_hs import Footprint2

## Adding external data to footprint
The input data for the footprint is the following:

- `nside` (int): Heapix NSIDE
- `nest` (bool): If ordering is nested (default=False)
- `pixel` (array): Pixels inside the footprint
- `detfrac_vals` (array, None): Detection fraction, if None is set to 1
- `zmax_vals` (array, None): Zmax, if None is set to 99

Just like the `ClCatalog` object, data in `Footorint` can be added from columns, a dictionary or a table.
First, let's create some input data:

In [ ]:
# Random values for detfrac and zmax for ftpt1
set_pixels1 = list(set(pixels1))
set_pixels2 = list(set(pixels2))
detfrac_rand = 0.9+.1*np.random.rand(len(set_pixels1))
z_rand = 0.5+.5*np.random.rand(len(set_pixels1))

### From columns<a id='from_cols'/>
To create a footprint fom columns, you have to pass the name as the initial argument and the data columns for the table as keyword arguments:

In [ ]:
ftpt1 = Footprint(nside=nside, pixel=set_pixels1, detfrac=detfrac_rand, zmax=z_rand)
ftpt1[:5]

In [ ]:
ftpt2 = Footprint2(nside=nside, pixel=set_pixels1, detfrac=detfrac_rand, zmax=z_rand)
ftpt2.data[hp.ring2nest(32, set_pixels1)][:2]

In [ ]:
from clevar.cosmology import AstroPyCosmology
cosmo = AstroPyCosmology()

In [ ]:
%%time
c1.add_ftpt_coverfrac(ftpt2, 1, 'mpc', cosmo=cosmo, window='nfw2D')

In [ ]:
import sys

In [ ]:
print('Size of Original HEALPix map [KB] ', sys.getsizeof(ftpt1.data)/1024.)
print('Size of Original HEALPix map [KB] ', sys.getsizeof(ftpt2[:])/1024.)

In [ ]:
ftpt1.data

In [ ]:
ftpt2[:]

In [ ]:
c

In [ ]:
import healsparse as hs

In [ ]:
dat = Footprint(nside=nside, pixel=set_pixels1, detfrac=detfrac_rand, zmax=z_rand)

In [ ]:
dtype = [('detfrac','f8'),('zmax','f8')]
hsp_map = hs.HealSparseMap.make_empty(
    nside, nside,
    dtype=dtype, primary='detfrac')
hsp_map.update_values_pix(
    np.array(set_pixels1),
    np.array([*zip(*[detfrac_rand, z_rand])], dtype=dtype),
    nest=True)
hsp_map.write('hsp_map.fits')

In [ ]:
#dtype = [('detfrac','f8'),('zmax','f8')]
hp_aux = np.full(hp.nside2npix(nside), hp.UNSEEN)
hp_aux[set_pixels1] = detfrac_rand
hs.HealSparseMap(nside_coverage=nside, healpix_map=hp_aux).write('hsp_map2.fits')

In [ ]:
detfrac_rand.size, np.array(set_pixels1).size

In [ ]:
ftpt_hs = Footprint.read_healsparse(
    'hsp_map.fits',
    full=False, tags={'pixel':'pixel'},
)

In [ ]:
ftpt_hs = Footprint.read_healsparse(
    'hsp_map2.fits',
    #full=False, tags={'pixel':'pixel'},
)

In [ ]:
ftpt_hs

In [ ]:
print(ftpt_hs.__repr__())

### From data table
You can also create a `ClCatalog` passing directly a full data table:

In [ ]:
from astropy.table import Table
ap_table = Table([set_pixels1, detfrac_rand], names=['hpix', 'df'])
ftpt1 = Footprint(nside=32, data=ap_table, tags={'pixel': 'hpix', 'detfrac':'df'})
ftpt1[:3]

The data table can also be a dictionary or a `numpy` array with names:

In [ ]:
# default colnames
print('default colnames:')
ftpt1 = Footprint(nside=32, data={'pixel': set_pixels1, 'detfrac': detfrac_rand})
display(ftpt1[:3])


print('different colnames:')
# different colnames
ftpt1 = Footprint(nside=32, data={'hpix': set_pixels1, 'df': detfrac_rand},
                  tags={'pixel': 'hpix', 'detfrac':'df'})
display(ftpt1[:3])

In [ ]:
# default colnames
print('default colnames:')
np_table = np.array(list(zip(set_pixels1, detfrac_rand)),
                    dtype=[('pixel', 'f4'), ('detfrac', 'f4')])
ftpt1 = Footprint(nside=32, data=np_table)
display(ftpt1[:3])


print('different colnames:')
# different colnames
np_table = np.array(list(zip(set_pixels1, detfrac_rand)),
                    dtype=[('hpix', 'f4'), ('df', 'f4')])
ftpt1 = Footprint(nside=32, data=np_table,
                  tags={'pixel': 'hpix', 'detfrac':'df'})
display(ftpt1[:3])

### Read the footprint from fits file
To read the footprint, the arguments are:

- `filename` (str): Name of `.fits` catalog
- `nside` (int): Heapix NSIDE
- `tags` (dict): Dictionary with the tags and column names. It must contain
  - `pixel` (str): Name of pixels column inside the footprint
  - `detfrac` (str): Name of detection fraction column, if None is set to 1
  - `zmax` (str): Name of Zmax column, if None is set to 99
- `nest` (bool): If ordering is nested (default=False)
- `full` (bool): read all columns of file

Let's first create a file with the footprint info:

In [ ]:
Footprint(nside=nside, pixel=set_pixels1, detfrac=detfrac_rand, zmax=z_rand).write('ftpt_temp.fits')

and then read it:

In [ ]:
Footprint.read('ftpt_temp.fits', nside=nside,
               tags={'pixel': 'pixel',
                     'detfrac': 'detfrac',
                     'zmax': 'zmax'})[:3]

In [ ]:
Footprint.read('ftpt_temp.fits', nside=nside,
               tags={'pixel': 'pixel'})[:3]

In [ ]:
Footprint.read('ftpt_temp.fits', nside=nside,
               tags={'pixel': 'pixel'}, full=True)[:3]

### Plotting the footprint
The footprints have an inbuilt function to plot their values

In [ ]:
f = ftpt1.plot('detfrac', bad_val=np.nan, auto_lim=True)
f = ftpt1.plot('zmax', bad_val=np.nan, auto_lim=True)

Clusters can also be added to the plot with their actual angular size:

In [ ]:
# increase cluster radius for display
c1['radius'] *= 10
f = ftpt1.plot('detfrac', bad_val=np.nan,
               ra_lim=[3, 8], dec_lim=[10, 15],
               cluster=c1)
# return original value
c1['radius'] /= 10

## Use ClEvaR functions to create a footprint
Import `create_footprint` functions to create a footprint based on a cluster catalog.
It can create a footprint based on cluster positions with a given `NSIDE`, or compute the best `NSIDE` based on a cluster density per pixel. It also can fill holes in the footprint.

In [ ]:
from clevar.footprint import create_artificial_footprint

Fixed `NSIDE`:

In [ ]:
ftpt1 = create_artificial_footprint(c1['ra'], c1['dec'], nside=64)

`NSIDE` from density:

In [ ]:
ftpt1 = create_artificial_footprint(c1['ra'], c1['dec'], nside=None, min_density=4)

- there is also an option to fill holes in this artificial footprint

In [ ]:
ftpt1 = create_artificial_footprint(c1['ra'], c1['dec'], nside=64, neighbor_fill=5)

In [ ]:
ftpt1['detfrac'] = np.random.rand(ftpt1.size)
f = ftpt1.plot('detfrac', np.nan,  latra=[5, 35], lonra=[-5, 40])

## Footprint masks
Add masks to clusters regarding the footprint. The `ClCatalog` object has has 3 functions related to the footprint:
- `add_ftpt_masks`: info for cluster in footprint
- `add_ftpt_coverfrac`: computes cover fraction
- `add_ftpt_coverfrac_nfw2D`: computes cover fraction weighted by a project NFW profile

In [ ]:
ftpt1 = Footprint(nside=nside, pixel=set_pixels1, detfrac=detfrac_rand, zmax=z_rand)
ftpt2 = Footprint(nside=nside, pixel=set_pixels2)
ftpt2['detfrac'][::2] = .5 # add effects to this footprint

In [ ]:
%%time
c1.add_ftpt_masks(ftpt1, ftpt2)
c2.add_ftpt_masks(ftpt2, ftpt1)

In [ ]:
display(c1[:4])
display(c2[:4])

Add coverfraction values based on the footprint. It needs a cosmology object.

In [ ]:
from clevar.cosmology import AstroPyCosmology
cosmo = AstroPyCosmology()

In [ ]:
%%time
c1.add_ftpt_coverfrac(ftpt2, 1, 'mpc', cosmo=cosmo, window='flat')
c1.add_ftpt_coverfrac(ftpt2, 1, 'mpc', cosmo=cosmo, window='nfw2D')
c2.add_ftpt_coverfrac(ftpt1, 1, 'mpc', cosmo=cosmo, window='nfw2D')

In [ ]:
display(c1[:4])
display(c2[:4])

## Saving and loading footprint quantities
`ClEvaR` has internal functions to save and load these quantities into the catalog so you don't have to compute them again:

In [ ]:
c1.save_footprint_quantities('cat1_ft_quantities.fits', overwrite=True)
c1.load_footprint_quantities('cat1_ft_quantities.fits')

## Application of footprint flags on recovery rate

The recovery rate of clusters should take into account the footprint of the catalogs. Regions without overlaps should not be taken into consideration. Here we show how this can be done with `ClEvaR`.

### Match catalogs
Let's match the catalogs to compute the recovery rate

In [ ]:
from clevar.match import ProximityMatch

In [ ]:
match_config = {
    'type': 'cross', # options are cross, cat1, cat2
    'which_radius': 'max', # Case of radius to be used, can be: cat1, cat2, min, max
    'preference': 'angular_proximity', # options are more_massive, angular_proximity or redshift_proximity
    'catalog1': {'delta_z':.2,
                'match_radius': '1 mpc'},
    'catalog2': {'delta_z':.2,
                'match_radius': '10 arcsec'}
}

In [ ]:
%%time
mt = ProximityMatch()
mt.match_from_config(c1, c2, match_config, cosmo=cosmo)

### Recovery rate

Use pass the parameters `mask` (masks all clusters) or `mask_unmatched` (masks only unmatched clusters) to consider only specific clusters on the recovery rate.
This way, you can exclude clusters outside the common regions from the consideration.

In [ ]:
from clevar.match_metrics import recovery

In [ ]:
zbins = np.linspace(0, 2, 11)
mbins = np.logspace(13, 14, 5)

Mask based on footprint overlap

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(15, 5))
recovery.plot(c1, 'cross', zbins, mbins, ax=axes[0], add_legend=False, shape='line')
recovery.plot(c1, 'cross', zbins, mbins, ax=axes[1], add_legend=False,
              mask=c1['ft_other'], shape='line')
recovery.plot(c1, 'cross', zbins, mbins, ax=axes[2],
              mask_unmatched=~c1['ft_other'], shape='line')
for ax in axes:
    ax.set_ylim(-.01, 1.05)
axes[0].text(1, 1.1, 'no mask')
axes[1].text(1, 1.1, 'mask all')
axes[2].text(1, 1.1, 'mask unmatched')
plt.show()

Mask based on coverfraction

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(15, 5))
recovery.plot(c1, 'cross', zbins, mbins, ax=axes[0], add_legend=False, shape='line')
recovery.plot(c1, 'cross', zbins, mbins, ax=axes[1], add_legend=False,
              mask=c1['cf_nfw_1_mpc']>.8, shape='line')
recovery.plot(c1, 'cross', zbins, mbins, ax=axes[2],
              mask_unmatched=c1['cf_nfw_1_mpc']<.8, shape='line')
for ax in axes:
    ax.set_ylim(-.01, 1.05)
axes[0].text(1, 1.1, 'no mask')
axes[1].text(1, 1.1, 'mask all')
axes[2].text(1, 1.1, 'mask unmatched')
plt.show()

You can check the exact numbers used on the 2D plots

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(20, 5))

recovery.plot2D(c1, 'cross', zbins, mbins, ax=axes[0],
                add_num=True, num_kwargs={'fontsize':12
                                         })
recovery.plot2D(c1, 'cross', zbins, mbins, ax=axes[1],
                add_num=True, num_kwargs={'fontsize':12},
               mask=c1['cf_nfw_1_mpc']>.8)
recovery.plot2D(c1, 'cross', zbins, mbins, ax=axes[2],
                add_num=True, num_kwargs={'fontsize':12},
               mask_unmatched=c1['cf_nfw_1_mpc']<.8)
axes[0].text(1, mbins[-1]*1.1,'no mask')
axes[1].text(1, mbins[-1]*1.1,'mask all')
axes[2].text(1, mbins[-1]*1.1,'mask unmatched')
    
plt.show()